In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import os
from matplotlib import image
from matplotlib import pyplot
from PIL import Image
from tqdm import tqdm
import cv2 
import pytesseract

## Extracting tar files

In [ ]:
import tarfile
path_tgz = "/content/drive/MyDrive/Colab Notebooks/pan_card_images/Pan Card Data/"
path_store= "/content/drive/MyDrive/Colab Notebooks/pan_card_images/extracted file/"

def extract(tar_add, ext_add):
    filename = tar_add
    tf = tarfile.open(filename)
    tf.extractall(ext_add)

import os
for file in os.listdir(path_tgz):
    if file.endswith(".tgz"):
        tgz_loc = (os.path.join(path_tgz, file))
        ext_loc = (os.path.splitext((os.path.join(path_store, file)))[0])
        extract(tgz_loc, ext_loc)
        

In [ ]:
tf.close()

## Creating Dataset

In [ ]:
import os

path_store= "/content/drive/MyDrive/Colab Notebooks/pan_card_images/extracted file/"

dates_img_path = []
dates_txt_path = []
names_img_path = []
names_txt_path = []
numbers_img_path = []
numbers_txt_path = []

for file in tqdm(os.listdir(path_store)):
    for img in os.listdir(os.path.join(path_store, file+'/')):
        if 'date' in file:
            if img.endswith(".png"):
                dates_img_path.append((os.path.join(path_store, file+'/',img)))
            else:
                dates_txt_path.append((os.path.join(path_store, file+'/',img)))
        if 'name' in file:
            if img.endswith(".png"):
                names_img_path.append((os.path.join(path_store, file+'/',img)))
            else:
                names_txt_path.append((os.path.join(path_store, file+'/',img)))
        if 'number' in file:
            if img.endswith(".png"):
                numbers_img_path.append((os.path.join(path_store, file+'/',img)))
            else:
                numbers_txt_path.append((os.path.join(path_store, file+'/',img)))
        
    

## Creating Dataframe

In [ ]:
dates_img_path.sort()
dates_txt_path.sort()
names_img_path.sort()
names_txt_path.sort()
numbers_img_path.sort()
numbers_txt_path.sort()
print("length of dates_img_path = ",len(dates_img_path))
print("length of dates_txt_path = ",len(dates_txt_path))
print("length of names_img_path = ",len(names_img_path))
print("length of names_txt_path = ",len(names_txt_path))
print("length of numbers_img_path = ",len(numbers_img_path))
print("length of numbers_txt_path = ",len(numbers_txt_path))

#### Extracting Text

In [ ]:

dates = []
names = []
numbers = []

def ext_txt(name,path):
  for file in tqdm(path):
    with open(file, 'rt') as fd:
      first_line = fd.readline()
      name.append(first_line)

ext_txt(dates, dates_txt_path)
ext_txt(names, names_txt_path)
ext_txt(numbers, numbers_txt_path)

In [ ]:
from PIL import Image

image = Image.open(dates_img_path[0])
print(image.format)
print(image.size)
print(image.mode)
# show the image
image.show()

In [ ]:
from matplotlib import image
from matplotlib import pyplot
# load image as pixel array
image = image.imread(dates_img_path[0])
# summarize shape of the pixel array
print(image.dtype)
print(image.shape)
# display the array of pixels as an image
pyplot.imshow(image)
pyplot.show()

# EDA


Image Preprocessing

In [ ]:
import numpy as np


# greyscalling
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

In [ ]:
from matplotlib import image
from matplotlib import pyplot
# load image as pixel array
image = image.imread(dates_img_path[0])
# summarize shape of the pixel array
#print(image.dtype)
#print(image.shape)
# display the array of pixels as an image
def plotting(im):
  pyplot.imshow(im)
  pyplot.show()

print("grayscale :")
plotting(get_grayscale(image))



In [ ]:
print("remove_noise :")
plotting(remove_noise(image))



In [ ]:
print("thresholding :")
plotting(thresholding(image))



In [ ]:
print("dilation :")
plotting(dilate(image))



In [ ]:
print("erosion :")
plotting(erode(image))



In [ ]:
print("opening :")
plotting(opening(image))



In [ ]:
print("canny edge detection :")
plotting(canny(image))



In [ ]:
print("deskew :")
plotting(deskew(image))



In [ ]:
print("match_template :")
plotting(match_template(image))


In [ ]:
!pip install opencv

# 01. Tesseract

In [ ]:
!sudo apt install tesseract-ocr

In [ ]:
!pip install pytesseract

In [ ]:
try:
 from PIL import Image
except ImportError:
 import Image
import cv2
import pytesseract

In [ ]:
!which tesseract

In [ ]:
pytesseract.pytesseract.tesseract_cmd = (
    r'/usr/bin/tesseract'
)

# opencv text

In [ ]:
import cv2 
import pytesseract

img = cv2.imread(dates_img_path[1])

# Adding custom options
custom_config = r'--oem 3 --psm 6 '
text = pytesseract.image_to_string(img)#, config=custom_config)
text[:-2]

In [ ]:
from matplotlib import image
from matplotlib import pyplot
# load image as pixel array
image = image.imread(names_img_path[5])
# summarize shape of the pixel array
#print(image.dtype)
#print(image.shape)
# display the array of pixels as an image
pyplot.imshow(image)
pyplot.show()

In [ ]:
import numpy as np

def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

"""# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated"""

In [ ]:
import cv2 
import pytesseract

img = cv2.imread(names_img_path[9])

#gray = get_grayscale(img)


def print_text(im):
  # Adding custom option
  custom_config = r'--oem 3 --psm 6 '
  text = pytesseract.image_to_string(im)#, config=custom_config)
  return (text)

print("after greyscalling : ", print_text(img))



# main text_extraction

In [ ]:
import cv2 
import pytesseract
from tqdm import tqdm


dates_txt_01 = [] # storing all the extracted dates 
names_txt_01 = [] # storing all the extracted names 
numbers_txt_01 = [] # storing all the extracted numbers 


def img_to_txt(pictures,new_list):
  for img in tqdm(pictures):
    image = cv2.imread(img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    custom_config = r'--oem 3 --psm 6 '
    text = pytesseract.image_to_string(image)#, config=custom_config)
    
    new_list.append(text[:-2])




In [ ]:
img_to_txt(names_img_path, names_txt_01)
df = pd.DataFrame({'names_Extracted_Text': names_txt_01})
df.to_csv("names_txt_01")
df.tail()

In [ ]:
df1 = pd.read_csv("/content/numbers_txt_01.csv")
df1

In [ ]:
img_to_txt(numbers_img_path, numbers_txt_01)
df1 = pd.DataFrame({'numbers_Extracted_Text': numbers_txt_01})
df1.to_csv("numbers_txt_01")
df1.tail()

In [ ]:
img_to_txt(dates_img_path, dates_txt_01)
df2 = pd.DataFrame({'dates_Extracted_Text': dates_txt_01})
df2.to_csv("dates_txt_01.csv")
df2.tail()

# Creating Datasets for DATES, NAMES, NUMBERS

In [ ]:
# merging oringinal dates with extracted dates in a dataframe

# temp df to load dates 
temp_df = pd.read_csv("/content/original_dates_text.csv").drop(['Unnamed: 0'],axis=1)
date_df = pd.concat([temp_df, df2], axis=1, join='inner')
date_df.to_csv("date_df.csv")
date_df.tail()

In [ ]:
# merging oringinal names with extracted names in a dataframe

# temp df to load names 
temp_df = pd.read_csv("/content/original_names_text.csv").drop(['Unnamed: 0'],axis=1)

name_df = pd.concat([temp_df, df], axis=1, join='inner')
name_df.tail()

In [ ]:
# merging oringinal numbers with extracted numbers in a dataframe

# temp df to load numbers
temp_df = pd.read_csv("/content/original_number_text.csv").drop(['Unnamed: 0'],axis=1)
df1 = pd.read_csv("/content/numbers_txt_01.csv").drop(['Unnamed: 0'],axis=1)
numbers_df = pd.concat([temp_df, df1], axis=1, join='inner')
numbers_df.to_csv("numbers_df.csv")
numbers_df.tail()

In [ ]:
import os

path_store= "/content/drive/MyDrive/Colab Notebooks/pan_card_images/extracted file/"

for file in os.listdir(path_store):
    for img in tqdm(os.listdir(os.path.join(path_store, file+'/'))):
        if 'name' in file:
            if img.endswith(".png"):
              try:
                #print(filename)
                #f.append(filename)
                img = Image.open(path_store + file +'/' + img)
                text = pytesseract.image_to_string(img, lang = 'eng')
                #t.append(text)
                print(text)
                #print('-='*20)
              except:
                continue
              #names_img_path.append((os.path.join(path_store, file+'/',img)))
        
    